In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

(1) dataset(영문 -> 한글 번역 쌍)

In [2]:
# 각 문장에 <sos> (start of sentence), <eos> (end of sentence) 토큰을 추가하여
# 각 문장 시작과 끝에 모델이 명시적으로 학습할 수 있도록 한다.
data_pairs = [
    ("<sos> i love you <eos>","<sos> 나는 너를 사랑해 <eos>"),
    ("<sos> he is a student <eos>", "<sos> 그는 학생이다 <eos>"),
    ("<sos> we are friends <eos>", "<sos> 우리는 친구다 <eos>"),
    ("<sos> she is kind <eos>","<sos> 그녀는 친절하다 <eos>"),
    ("<sos> he is a awesome <eos>", "<sos> 그는 멋지다 <eos>")
]

(2) 단어 사전 생성

In [7]:
# 단어 사전(vocabulary) 생성 함수
# 주어진 문장 집합을 입력받아 단어별로 고유한 인덱스를 부여하는 함수.
def build_vocab(sentences):
    vocab = {"<pad>": 0} # 패딩 토큰을 0번 인덱스로 지정
    for sent in sentences:
        for word in sent.split():
            if word not in vocab:
                vocab[word] = len(vocab)
    return vocab

In [8]:
# 영문(source)과 한글(target) 각각의 단어 사전 생성
src_vocab = build_vocab([p[0] for p in data_pairs])
trg_vocab = build_vocab([p[1] for p in data_pairs])

In [9]:
# 단어 개수 확인
src_vocab_size = len(src_vocab)
trg_vocab_size = len(trg_vocab)

In [10]:
# 작업 확인용 출력
print("SRC vocab:", src_vocab)
print("TRG vocab:", trg_vocab)

SRC vocab: {'<pad>': 0, '<sos>': 1, 'i': 2, 'love': 3, 'you': 4, '<eos>': 5, 'he': 6, 'is': 7, 'a': 8, 'student': 9, 'we': 10, 'are': 11, 'friends': 12, 'she': 13, 'kind': 14, 'awesome': 15}
TRG vocab: {'<pad>': 0, '<sos>': 1, '나는': 2, '너를': 3, '사랑해': 4, '<eos>': 5, '그는': 6, '학생이다': 7, '우리는': 8, '친구다': 9, '그녀는': 10, '친절하다': 11, '멋지다': 12}


(4) 인코더(Encoder) 정의

In [18]:
# 인코더 함수 정의
# 입력 문장을 벡터로 임베딩 후 LSTM을 통해 문맥 벡터(hidden, cell) 생성
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim) # 단어 -> 임베딩 벡터
        self.rnn = nn.LSTM(emb_dim, hid_dim) # LSTM으로 문맥 인코딩

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

(5) 디코더(Decoder) 정의

In [52]:
# 디코더 함수 정의
# 인코더에서 전달받은 문맥 벡터를 기반으로 한 단어씩 번역을 생성
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim)
        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, (hidden, cell) = self.rnn(embedded, (hidden,cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

(6) 인코더-디코더 결합한 Seq2Seq 정의

In [53]:
# 인코더-디코더 결합 모델
# Encoder의 출력을 디코더에 전달하여 문장 전체를 번역
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.1):
        trg_len, batch_size = trg.shape
        output_dim = self.decoder.fc_out.out_features
        outputs = torch.zeros(trg_len, batch_size, output_dim)

        hidden, cell = self.encoder(src)
        input = trg[0,:]

        # 타임스텝별로 단어 예측 수행
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            top1 = output.argmax(1)
            input = trg[t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

(7) 모델 초기화

In [54]:
INPUT_DIM = src_vocab_size # 입력 단어 수
OUTPUT_DIM = trg_vocab_size # 출력 단어 수
EMB_DIM = 32 # 임베딩 벡터 차원
HID_DIM = 64 # LSTM 은닉 상태 차원

- 모델 객체 생성

In [55]:
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM)
model = Seq2Seq(enc, dec)

(8) 학습 준비 (손실함수 및 옵티마이저)

In [56]:
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab["<pad>"])
optimizer = optim.Adam(model.parameters(), lr=0.01)

(9) 학습 데이터 텐서 변환

In [57]:
# 문장을 숫자 텐서로 변환하는 함수
# 각 단어를 사전에 따라 인덱스로 바꾸고, (길이, 배치) 형태로 reshape.
def tensorize(sentence, vocab):
    idxs = [vocab[w] for w in sentence.split()]     # 각 단어를 숫자로 변환
    return torch.tensor(idxs, dtype=torch.long).unsqueeze(1)  # (seq_len, batch=1)

In [58]:
src_tensors = [tensorize(src, src_vocab) for src, _ in data_pairs]
trg_tensors = [tensorize(trg, trg_vocab) for _, trg in data_pairs]

(10) 학습 수행

In [61]:
# 각 에폭마다 모든 문장을 학습하고 손실 값 출력
for epoch in range(300):
    total_loss = 0
    for src_tensor, trg_tensor in zip(src_tensors, trg_tensors):
        optimizer.zero_grad()
        output = model(src_tensor, trg_tensor)
        output_dim = output.shape[-1]
        # <sos> 제외 후 손실 계산
        loss = criterion(output[1:].view(-1, output_dim), trg_tensor[1:].view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {total_loss:3f}")

Epoch 0 | Loss: 12.328947
Epoch 50 | Loss: 0.011107
Epoch 100 | Loss: 0.003504
Epoch 150 | Loss: 0.001790
Epoch 200 | Loss: 0.001103
Epoch 250 | Loss: 0.000749


(11) 번역기

In [64]:
# 번역 함수 정의
# 학습된 모델을 이용해 새로운 문장을 번역
def translate(sentence):
    model.eval() 

    sentence = "<sos> " + sentence + " <eos>"

    src_idx = torch.tensor([[src_vocab[w] for w in sentence.split()]], dtype=torch.long).T

    hidden, cell = model.encoder(src_idx)

    input = torch.tensor([trg_vocab["<sos>"]])
    result = []

    for _ in range(10):
        output, hidden, cell = model.decoder(input, hidden, cell)
        top1 = output.argmax(1)
        word = [k for k, v in trg_vocab.items() if v == top1.item()][0]
        if word == "<eos>":
            break
        result.append(word)
        input = top1
    return " ".join(result)

### ---- 번역기 테스트 ----

In [65]:
print(translate("i love you"))

나는 너를 사랑해


In [66]:
print(translate("we are friends"))

우리는 친구다


In [67]:
print(translate("i am awesome"))

KeyError: 'am'